In [13]:
import pandas as pd
from lib.preprocess import preprocessor
from lib.utils import *

def get_repo_name(i):
    """Get id from lookup... then get name from bigquery table? """
    pass
    
def predict_ai(model, pos, X):
    """ pos is the positive class vectors, X is the entire dataset """    
    model.fit(pos)
    preds = model.predict(X)
    return np.argwhere(preds == 1)

In [14]:
size = int(5472100/2)
embeddings = get_embeddings('./ss/ss_repo_embeds.txt', size)

NameError: name 'np' is not defined

In [11]:
repos = [
    'GuessWhatGame/vqa',
    'gitlimlab/SSGAN-Tensorflow',
    'mufarhan/sequence_class_NIPS_2017',
    'KeiraZhao/SPN',
    'ap229997/Conditional-Batch-Norm'
]

write_ai_readmes(repos, 'prepared-readmes/ai_readmes.txt')

In [ ]:
from sklearn.svm import OneClassSVM

vecs = get_ss_embed('ss/ai-vectors.txt')
svm = OneClassSVM()
pos = predict_ai(svm, vecs, embeddings)

for p in pos: 
    print(get_sentence(p[0]))